# 9.1 QR-Gram 파이썬 코딩

In [48]:
### 행렬 선언
A = [[1,0,1],[0,1,1],[1,2,0]]

print(f"{A}")

### 열 벡터 분해
n = len(A)
m = len(A[0])
a1 = []
a2 = []
a3 = []

for i in range(0, n):
    a1.append(A[i][0])
    a2.append(A[i][1])
    a3.append(A[i][2])
    
### u1
u1 = a1

print(f"u1 : {u1}")

### norm u1
norm_u1 = 0
for i in range(0, len(u1)):
    norm_u1 += u1[i]**2
norm_u1 = norm_u1**(1/2)

### u2
u2 = []

dp21 = 0 # dot product of a2, u1
for i in range(0, len(u1)):
    temp = a2[i]*u1[i]
    dp21 += temp

for i in range(0, n):
    temp = a2[i] - (dp21 / norm_u1**2)*u1[i]
    u2.append(temp)
    
print(f"u2 : {u2}")

### norm u2
norm_u2 = 0
for i in range(0, n):
    norm_u2 += u2[i]**2
norm_u2 = norm_u2**(1/2)

### u3
u3 = []
dp31 = 0 # dot product of a3, u1
for i in range(0, n):
    temp = a3[i]*u1[i]
    dp31 += temp
    
dp32 = 0 # dot product of a3, u2
for i in range(0, n):
    temp = a3[i]*u2[i]
    dp32 += temp
    
for i in range(0, n):
    temp = a3[i] - (dp31 / norm_u1**2) * u1[i] - (dp32 / norm_u2**2) * u2[i]
    u3.append(temp)
    
print(f"u3 : {u3}")

### norm u3
norm_u3 = 0
for i in range(0, n):
    norm_u3 += u3[i]**2
norm_u3 = norm_u3**(1/2)

# v
v1 = []
v2 = []
v3 = []
for i in range(0, n):
    temp1 = u1[i] / norm_u1
    temp2 = u2[i] / norm_u2
    temp3 = u3[i] / norm_u3
    v1.append(temp1)
    v2.append(temp2)
    v3.append(temp3)
    
# Q matrix
Q = [[0,0,0],[0,0,0],[0,0,0]]
for i in range(0, n):
    Q[i][0] = v1[i]
    Q[i][1] = v2[i]
    Q[i][2] = v3[i]

print(f"Q : {Q}")

# R matrix
R = [[0,0,0],[0,0,0],[0,0,0]]
for i in range(0 ,n):
    R[0][0] += a1[i]*v1[i]
    R[0][1] += a2[i]*v1[i]
    R[0][2] += a3[i]*v1[i]
    R[1][1] += a2[i]*v2[i]
    R[1][2] += a3[i]*v2[i]
    R[2][2] += a3[i]*v3[i]
    
print(f"R : {R}")

def m_mul(A, B):

    n = len(A)
    m = len(A[0])
    l = len(B[0])
    
    C = [] # result
    
    for i in range(0, n):
        row = []
        for j in range(0, l):
            temp = 0
            for k in range(0, m):
                temp += A[i][k] * B[k][j]
            row.append(temp)
        C.append(row)
        
    return C
print(f"검산 : QR = {m_mul(Q, R)}")

[[1, 0, 1], [0, 1, 1], [1, 2, 0]]
u1 : [1, 0, 1]
u2 : [-0.9999999999999998, 1.0, 1.0000000000000002]
u3 : [0.5000000000000002, 0.9999999999999999, -0.49999999999999994]
Q : [[0.7071067811865475, -0.5773502691896256, 0.40824829046386324], [0.0, 0.5773502691896258, 0.816496580927726], [0.7071067811865475, 0.577350269189626, -0.408248290463863]]
R : [[1.414213562373095, 1.414213562373095, 0.7071067811865475], [0, 1.7320508075688776, 2.220446049250313e-16], [0, 0, 1.2247448713915894]]
검산 : QR = [[0.9999999999999998, -2.220446049250313e-16, 1.0000000000000002], [0.0, 1.0000000000000002, 1.0000000000000004], [0.9999999999999998, 2.0, -1.1102230246251565e-16]]


# 9.2 QR-Gram 파이썬 함수 생성

In [25]:
### tp
def tp(A):
    n = len(A)
    m = len(A[0])
    At = [] # result matrix
    for i in range(0, m):
        row = []
        for j in range(0, n):
            temp = A[j][i]
            row.append(temp)
        At.append(row)
    return At

### ip
def ip(a, b):
    n = len(a)
    res = 0 # result scalar
    for i in range(0, n):
        res += a[i]*b[i]
    return res

### norm
def norm(a):
    n = len(a)
    res = 0 # result scalar
    for i in range(0, n):
        res += a[i]*a[i]
    res = res**(1/2)
    return res

### normalize
def normalize(a):
    """to make vector a norm 1"""
    n = len(a)
    res = [] # result vector
    for i in range(0, n):
        temp = a[i] / norm(a)
        res.append(temp)
    return res

### qr_gram
def qr_g(A):
    n = len(A)
    m = len(A[0])
    Q = [] # result matrix Q
    R = [] # result matrix R
    
    At = tp(A)
    norm_list = []
    
    U = [] # U matrix
    V = [] # normalized
    
    for i in range(0, n):
        if i == 0:
            u = At[i]
            norm_u = norm(u)
            U.append(u)
            norm_list.append(norm_u)
        else:
            a = At[i]
            dp_list = [] # dot product list
            for j in range(0, i):
                dp = ip(a, U[j])
                dp_list.append(dp)
                
            u = []
            for j in range(0, n):
                temp = a[j]
                for k in range(0, i):
                    temp -= (dp_list[k] / norm_list[k]**2)*U[k][j]
                u.append(temp)
            norm_u = norm(u)
            U.append(u)
            norm_list.append(norm_u)
            
        v = normalize(u)
        V.append(v)
        
    Q = tp(V)
    
    for i in range(0, n):
        r = []
        for j in range(0, n):
            if i > j:
                r.append(0)
            else:
                r_ele = ip(At[j], V[i])
                r.append(r_ele)
        R.append(r)
    
    return Q, R

A = [[1,0,1],[0,1,1],[1,2,0]]
qr_g(A)

([[0.7071067811865475, -0.5773502691896256, 0.40824829046386324],
  [0.0, 0.5773502691896258, 0.816496580927726],
  [0.7071067811865475, 0.577350269189626, -0.408248290463863]],
 [[1.414213562373095, 1.414213562373095, 0.7071067811865475],
  [0, 1.7320508075688776, 2.220446049250313e-16],
  [0, 0, 1.2247448713915894]])

# 9.3 QR-H 파이썬 함수 생성

In [84]:
### zero
def zero(n, m):
    Z = [] # result matrix
    for i in range(0, n):
        row =[]
        for j in range(0, m):
            row.append(0)
        Z.append(row)
    return Z

### deepcopy
def deepcopy(A):
    if type(A[0]) == list:
        n = len(A)
        m = len(A[0])
        res = zero(n, m) # result matrix
        for i in range(0, n):
            for j in range(0, m):
                res[i][j] = A[i][j]
        return res
    else:
        n = len(A)
        res = [] # result vector
        for i in range(0, n):
            res.append(A[i])
        return res
    
### tp
def tp(A):
    n = len(A)
    m = len(A[0])
    At = [] # result matrix
    for i in range(0, m):
        row = []
        for j in range(0, n):
            temp = A[j][i]
            row.append(temp)
        At.append(row)
    return At

### matmul
def m_mul(A, B):
    n = len(A)
    m = len(A[0])
    l = len(B[0])
    res = [] # result matrix
    for i in range(0, n):
        row = []
        for j in range(0, l):
            temp = 0
            for k in range(0, m):
                temp += A[i][k] * B[k][j]
            row.append(temp)
        res.append(row)
    return res
    
### mat subtract
def m_sub(A, B):
    n = len(A)
    m = len(A[0])
    res = [] # result matrix
    for i in range(0, n):
        row = []
        for j in range(0, m):
            temp = A[i][j] - B[i][j]
            row.append(temp)
        res.append(row)
    return res

### identity
def iden(n):
    I = [] # result matrix
    for i in range(0, n):
        row = []
        for j in range(0, n):
            if i==j:
                row.append(1)
            else:
                row.append(0)
        I.append(row)
    return I
    
### ip
def ip(a, b):
    n = len(a)
    res = 0 # result scalar
    for i in range(0, n):
        res += a[i]*b[i]
    return res

### op
def op(a, b):
    n = len(a)
    m = len(b)
    res = [] # result matrix
    for i in range(0, n):
        row = []
        for j in range(0, m):
            temp = a[i]*b[j]
            row.append(temp)
        res.append(row)
    return res

### norm
def norm(a):
    n = len(a)
    res = 0 # result scalar
    for i in range(0, n):
        res += a[i] ** 2
    res = res **(1/2)
    return res

### vec add
def v_add(u, v):
    n = len(u)
    res = [] # result vector
    for i in range(n):
        temp = u[i] + v[i]
        res.append(temp)
    return res

### sign
def sign(a):
    res = 1 # result scalar
    if a < 0:
        res = -1
    return res

### hh
def hh(v):
    n = len(v)
    cp = op(v,v)
    dp = ip(v,v)
    T = [] # temp matrix
    for i in range(0, n):
        row = []
        for j in range(0, n):
            temp = (2/dp)*cp[i][j]
            row.append(temp)
        T.append(row)
    H = m_sub(iden(n), T)
    return H

### qr h
def qr_h(A):
    n = len(A)
    m = len(A[0])
    H_list = [] # H 행렬을 모아 담을 리스트 선언
    
    for i in range(0, m): # 행렬 A의 열에 대한 반복문
        if i == 0: # 첫 번째 열에 관하여
            A1 = deepcopy(A) # 1번째 행렬 A
            exA = A1
        elif i < m-1: # 마지막 열이 아닌 열에 관하여
            Ai = [] # i번째 행렬 A
            for j in range(1, len(exA)): # 행렬 exA의 0번째 행을 제외하고 반복문 수행
                row = []
                for k in range(1, len(exA[0])): # 행렬 exA의 0번째 열을 제외하고 반복문 수행
                    row.append(HA[j][k]) 
                Ai.append(row)
            exA = Ai
        elif i == m-1: # 마지막 열에 관하여
            Am = [] # m번째 행렬 A
            for j in range(1, len(HA)): 
                Am.append(HA[j][1])
            exA = Am
            
        # 열 추출
        if i < m-1: # 마지막 열이 아니라면
            a = tp(exA)[0]
        else: # 마지막 열이라면
            a = exA
        nm = norm(a)
        
        # e 생성
        e = [1]
        for j in range(0, len(a)-1):
            e.append(0)
        
        # v 생성
        temp_e = []
        for j in range(0,  len(e)):
            temp = sign(a[0]) * nm * e[j]
            temp_e.append(temp)
        v = v_add(a, temp_e)
        
        # H 생성
        H = hh(v)
        
        # H*A part
        if i == m-1: # i가 마지막 열이라면
            HA = []
            for j in range(0, len(H)):
                temp = 0
                for k in range(0, len(H[0])):
                    temp += H[j][k] * exA[k]
                HA.append(temp)
        else:
            HA = m_mul(H, exA)
                
        H_list.append(H)
        
        if i > 0:
            temp_H = iden(len(A))
            for j in range(i, len(A)):
                for k in range(i, len(A)):
                    temp_H[j][k] = H_list[-1][j-i][k-i]        
            H_list[-1] = temp_H
            
    Q = deepcopy(H_list[0])
    for j in range(0, len(H_list)-1):
        Q = m_mul(Q, H_list[j+1])
        
    R = deepcopy(H_list[-1])
    for j in range(1, len(H_list)):
        R = m_mul(R, H_list[-(j+1)])
    R = m_mul(R, A)
        
    return Q, R

A = [[1,-1,4],[1,4,-2],[1,4,2],[1,-1,0]]
Q, R = qr_h(A)

print(Q)
print(R)

[[-0.5, 0.5000000000000001, -0.49999999999999994, -0.5], [-0.5, -0.5000000000000001, 0.5000000000000002, -0.5000000000000001], [-0.5, -0.5000000000000001, -0.5000000000000001, 0.5], [-0.5, 0.5, 0.5, 0.5]]
[[-2.0, -3.0, -2.0], [-1.1102230246251565e-16, -5.000000000000001, 2.000000000000001], [1.1102230246251565e-16, -1.1102230246251565e-16, -4.0], [2.220446049250313e-16, 4.440892098500626e-16, 8.881784197001252e-16]]


# 9.4 QR분해 넘파이 실습

In [45]:
import numpy as np
A = np.array([[1,0,1],[0,1,1],[1,2,0]])
Q, R = np.linalg.qr(A)
print(Q)
print(R)
np.matmul(Q, R)

[[-0.70710678  0.57735027 -0.40824829]
 [-0.         -0.57735027 -0.81649658]
 [-0.70710678 -0.57735027  0.40824829]]
[[-1.41421356e+00 -1.41421356e+00 -7.07106781e-01]
 [ 0.00000000e+00 -1.73205081e+00  1.11022302e-16]
 [ 0.00000000e+00  0.00000000e+00 -1.22474487e+00]]


array([[ 1.00000000e+00, -5.20510307e-16,  1.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 1.00000000e+00,  2.00000000e+00, -9.83730272e-17]])

# 11.1 고윳값 분해 파이썬 실습

In [86]:
def eig_qr(A):
    n = len(A)
    E = deepcopy(A)
    V = iden(n)
    for i in range(0, 100):
        Q, R = qr_h(E)
        E = m_mul(R, Q)
        V = m_mul(V, Q)
    
    return E, V

A = [[3,2,1],[2,1,4],[1,4,2]]
E, V = eig_qr(A)
print(E)
print(V)

[[6.712979947116978, 3.8630226281360384e-16, 6.365463701241705e-16], [2.52839182138358e-40, -2.6675211316251697, 1.8823802983500261e-13], [1.0973214614231972e-53, 1.8802948195066644e-13, 1.954541184508183]]
[[0.4947304488047792, -0.1605961446739691, -0.854078837895704], [0.6079756458714625, 0.7661978248229938, 0.20810215583624259], [0.6209729438973982, -0.6222136060488971, 0.476699938530547]]


# 11.2 고윳값 분해 넘파이 실습

In [87]:
import numpy as np
A = np.array([[3,2,1],[2,1,4],[1,4,2]])
e, v = np.linalg.eig(A)
print(e)
print(v)

[ 6.71297995  1.95454118 -2.66752113]
[[ 0.49473045  0.85407884  0.16059614]
 [ 0.60797565 -0.20810216 -0.76619782]
 [ 0.62097294 -0.47669994  0.62221361]]


# 13.1 특이값 분해 파이썬 실습

In [98]:
def diag(A):
    n = len(A)
    ele = [] # result elements
    for i in range(0, n):
        row = []
        for j in range(0, n):
            if i==j:
                row.append(A[i][i])
            else:
                row.append(0)
        ele.append(row)
    return ele
    

def svd(A):
    At = tp(A)
    AtA = m_mul(At, A)
    E, V = eig_qr(AtA)
    n = len(AtA)
    for i in range(0, n):
        E[i][i] = E[i][i]**(1/2)
    S = diag(E)
    Vt = tp(V)
    
    AV = m_mul(A, V)
    AVt = tp(AV)
    Ut = []
    for vector in AVt:
        Ut.append(normalize(vector))
    U = tp(Ut)
    return U, S, Vt

B = [[3,6],[2,3],[1,2],[5,5]]
U, S, Vt = svd(B)
print(U)
print(S)
print(Vt)

[[0.6305881956601932, 0.6507005141176461], [0.3429460755308733, 0.07207639573280686], [0.21019606522006437, 0.2169001713725487], [0.6637500515540445, -0.7241188781987097]]
[[10.508040759929596, 0], [0, 1.606573803987343]]
[[0.5811362175448523, 0.8138063016822019], [-0.8138063016822016, 0.5811362175448519]]


# 13.2 특이값 분해 넘파이 실습

In [99]:
import numpy as np

B = np.array([[3,6],[2,3],[1,2],[5,5]])
U, S, Vt = np.linalg.svd(B)
print(U)
print(S)
print(Vt)

[[-0.6305882   0.65070051 -0.34404196  0.24613512]
 [-0.34294608  0.0720764   0.09856768 -0.93138466]
 [-0.21019607  0.21690017  0.9335582   0.19297931]
 [-0.66375005 -0.72411888 -0.01971354  0.18627693]]
[10.50804076  1.6065738 ]
[[-0.58113622 -0.8138063 ]
 [-0.8138063   0.58113622]]
